In [ ]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("/home/srinivasan/Downloads/haarcascade_frontalface_alt.xml")
skip=0
dataset_path=dataset_path='/home/srinivasan/dataset/'
face_data=[]
labels=[] 
class_id=0 
names={}
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4] 
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)        
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
#print(face_dataset.shape)
#print(face_labels.shape)

knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(face_dataset,face_labels)

while True:
    ret,frame=cap.read()
    if(ret==False):
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h=face
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100)) 
        out=knn.predict(face_section.flatten().reshape(-1,30000))
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("Faces",frame)
    key_pressed=cv2.waitKey(1) & 0xFF 
    if(key_pressed==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()